# Technical Analysis of Stock Data

This notebook performs technical analysis on historical stock data using various indicators.

In [6]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas_ta as ta

# Set plot style
plt.style.use('seaborn-v0_8')
sns.set_theme()

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

ImportError: cannot import name 'NaN' from 'numpy' (/home/abel/Downloads/tenx-stock-sentiment-analysis/.venv/lib/python3.12/site-packages/numpy/__init__.py)

In [ ]:
# Define data directory
data_dir = Path('data/yfinance_data')

# List of stock files
stock_files = {
    'AAPL': 'AAPL_historical_data.csv',
    'AMZN': 'AMZN_historical_data.csv',
    'GOOG': 'GOOG_historical_data.csv',
    'META': 'META_historical_data.csv',
    'MSFT': 'MSFT_historical_data.csv',
    'NVDA': 'NVDA_historical_data.csv',
    'TSLA': 'TSLA_historical_data.csv'
}

# Load all stock data into a dictionary
stock_data = {}
for ticker, file in stock_files.items():
    df = pd.read_csv(data_dir / file)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    stock_data[ticker] = df

# Display sample data for one stock
print(f"Sample data for AAPL:")
display(stock_data['AAPL'].head())

In [ ]:
# Function to add technical indicators
def add_technical_indicators(df):
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Trend Indicators
    df['SMA_20'] = ta.sma(df['Close'], length=20)
    df['SMA_50'] = ta.sma(df['Close'], length=50)
    df['EMA_20'] = ta.ema(df['Close'], length=20)
    
    # Momentum Indicators
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df['MACD'] = ta.macd(df['Close'])['MACD_12_26_9']
    df['MACD_Signal'] = ta.macd(df['Close'])['MACDs_12_26_9']
    
    # Volatility Indicators
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = ta.bbands(df['Close']).T
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'])
    
    # Volume Indicators
    df['OBV'] = ta.obv(df['Close'], df['Volume'])
    
    return df

# Add indicators to each stock's data
for ticker in stock_data:
    stock_data[ticker] = add_technical_indicators(stock_data[ticker])

# Display indicators for one stock
print("Technical Indicators for AAPL:")
display(stock_data['AAPL'].tail())

In [ ]:
# Plot price and moving averages for a stock
def plot_price_and_ma(ticker, df):
    plt.figure(figsize=(15, 8))
    plt.plot(df.index, df['Close'], label='Close Price')
    plt.plot(df.index, df['SMA_20'], label='20-day SMA')
    plt.plot(df.index, df['SMA_50'], label='50-day SMA')
    plt.title(f'{ticker} Price and Moving Averages')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot RSI for a stock
def plot_rsi(ticker, df):
    plt.figure(figsize=(15, 8))
    plt.plot(df.index, df['RSI'], label='RSI')
    plt.axhline(y=70, color='r', linestyle='--', label='Overbought')
    plt.axhline(y=30, color='g', linestyle='--', label='Oversold')
    plt.title(f'{ticker} RSI')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot Bollinger Bands for a stock
def plot_bollinger_bands(ticker, df):
    plt.figure(figsize=(15, 8))
    plt.plot(df.index, df['Close'], label='Close Price')
    plt.plot(df.index, df['BB_Upper'], label='Upper Band', linestyle='--')
    plt.plot(df.index, df['BB_Middle'], label='Middle Band')
    plt.plot(df.index, df['BB_Lower'], label='Lower Band', linestyle='--')
    plt.title(f'{ticker} Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot indicators for a specific stock (e.g., AAPL)
ticker = 'AAPL'
plot_price_and_ma(ticker, stock_data[ticker])
plot_rsi(ticker, stock_data[ticker])
plot_bollinger_bands(ticker, stock_data[ticker])

## Technical Analysis Insights

### Moving Averages
- 20-day SMA: Short-term trend indicator
- 50-day SMA: Medium-term trend indicator
- Golden Cross: When 20-day SMA crosses above 50-day SMA (bullish)
- Death Cross: When 20-day SMA crosses below 50-day SMA (bearish)

### RSI (Relative Strength Index)
- Above 70: Potentially overbought
- Below 30: Potentially oversold
- 50: Neutral level

### Bollinger Bands
- Upper Band: 2 standard deviations above 20-day SMA
- Lower Band: 2 standard deviations below 20-day SMA
- Price touching upper band: Potentially overbought
- Price touching lower band: Potentially oversold

### MACD (Moving Average Convergence Divergence)
- Positive MACD: Bullish momentum
- Negative MACD: Bearish momentum
- MACD crossing above signal line: Bullish signal
- MACD crossing below signal line: Bearish signal